컬럼 확인

In [2]:
import geopandas as gpd

gdf_admin = gpd.read_file("/home/kbj/Downloads/HangJeongDong_ver20241001.geojson")
print(gdf_admin.columns)  # 어떤 행정동 정보가 있는지 확인


Index(['adm_nm', 'adm_cd2', 'sgg', 'sido', 'sidonm', 'sggnm', 'adm_cd',
       'geometry'],
      dtype='object')


In [5]:
import pandas as pd
from shapely.geometry import Point

df_station = pd.read_csv("/home/kbj/Downloads/station_table.csv")

# geometry 컬럼 생성
df_station["geometry"] = df_station.apply(lambda row: Point(row["station_longitude"], row["station_latitude"]), axis=1)
gdf_station = gpd.GeoDataFrame(df_station, geometry="geometry", crs="EPSG:4326")


행정동 경계 좌표계 맞추기

In [ ]:
gdf_admin = gdf_admin.to_crs("EPSG:4326")  
gdf_joined = gpd.sjoin(gdf_station, gdf_admin, how="left", predicate="intersects")


In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

df = pd.read_csv("/home/kbj/Downloads/station_table.csv")

gdf_stations = gpd.GeoDataFrame(
    df,
    geometry=df.apply(lambda row: Point(row['station_longitude'], row['station_latitude']), axis=1),
    crs="EPSG:4326"
)

gdf_dong = gpd.read_file("/home/kbj/Downloads/HangJeongDong_ver20241001.geojson")

gdf_joined = gpd.sjoin(gdf_stations, gdf_dong, how="left", predicate="within")

df_result = gdf_joined[[
    "station_id", "station_gu", "station_dong",
    "station_latitude", "station_longitude", "adm_nm"
]].rename(columns={"adm_nm": "행정동명"})


df_result.to_csv("/home/kbj/Downloads/station_with_admin_dong.csv", index=False)
print("완료! 파일 저장됨: station_with_admin_dong.csv")


완료! 파일 저장됨: station_with_admin_dong.csv


In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

gdf_dong = gpd.read_file("/home/kbj/Downloads/HangJeongDong_ver20241001.geojson")

def get_dong_from_coords(lat, lon):
    point_gdf = gpd.GeoDataFrame(
        geometry=[Point(lon, lat)],
        crs="EPSG:4326"
    )

    joined = gpd.sjoin(point_gdf, gdf_dong, how="left", predicate="within")

    if not joined.empty:
        full_name = joined.iloc[0]["adm_nm"]  # ex. "서울특별시 강남구 역삼1동"
        parts = full_name.split()
        if len(parts) >= 3:
            return parts[1], parts[2] 
        else:
            return None, None
    else:
        return None, None

lat = 37.4981
lon = 127.0276
gu, dong = get_dong_from_coords(lat, lon)
print(f"✅ 위치: 서울특별시 {gu} {dong}" if gu and dong else "❌ 위치를 찾을 수 없습니다.")


✅ 위치: 서울특별시 강남구 역삼1동


In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

gdf_dong = gpd.read_file("/home/kbj/Downloads/HangJeongDong_ver20241001.geojson")

df_code = pd.read_csv("/home/kbj/Downloads/행정동코드_변환.csv")

df_code["행정동코드"] = df_code["행정동코드"].astype(str).str.slice(0, 8)
gdf_dong["adm_cd"] = gdf_dong["adm_cd"].astype(str)

gdf_dong = gdf_dong.merge(df_code, left_on="adm_cd", right_on="행정동코드", how="left")

def find_admin_dong(lat, lon):
    point = Point(lon, lat)  # 좌표 순서: 경도, 위도
    match = gdf_dong[gdf_dong.geometry.intersects(point)]
    if not match.empty:
        row = match.iloc[0]
        return row["시군구명"], row["읍면동명"]
    else:
        return None, None

lat = 37.4981
lon = 127.0276
gu, dong = find_admin_dong(lat, lon)

if gu and dong:
    print(f" 위치: 서울특별시 {gu} {dong}")
else:
    print(" 위치를 찾을 수 없습니다.")


✅ 위치: 서울특별시 nan nan


위도 경도 잘나오는지 파악

In [10]:
from shapely.geometry import Point

test_point = Point(127.0276, 37.4981)  # 경도, 위도
print(test_point)


POINT (127.0276 37.4981)


행이 매칭되는지

In [ ]:
point = Point(127.0276, 37.4981)
matches = gdf_dong[gdf_dong.geometry.contains(point)]

print("매칭된 행 개수:", len(matches))
if not matches.empty:
    print(matches[["시군구명", "읍면동명"]])
else:
    print(" 이 좌표를 포함하는 행정동이 없습니다.")


매칭된 행 개수: 1
    시군구명 읍면동명
364  NaN  NaN


In [4]:
print(df_code.columns)


Index(['행정동코드', '시도명', '시군구명', '읍면동명'], dtype='object')


In [10]:
print(gdf_dong.columns)


Index(['adm_nm', 'adm_cd2', 'sgg', 'sido', 'sidonm', 'sggnm', 'adm_cd',
       'geometry'],
      dtype='object')


In [ ]:
import pandas as pd

df = pd.read_csv("/home/kbj/Downloads/station_with_admin_dong.csv")

df["행정동_only"] = df["행정동명"].str.replace("서울특별시", "", regex=False)
df["행정동_only"] = df["행정동_only"].str.replace(r"\s*\S+구\s*", "", regex=True)  # 'XX구' 제거

print(df[["행정동명", "행정동_only"]].head())


df.to_csv("station_with_dong_only.csv", index=False)


             행정동명 행정동_only
0   서울특별시 마포구 서교동      서교동
1  서울특별시 광진구 자양3동     자양3동
2  서울특별시 양천구 신정7동     신정7동
3  서울특별시 양천구 신월3동     신월3동
4   서울특별시 양천구 목1동      목1동


In [ ]:
import pandas as pd

# CSV 불러오기
df = pd.read_csv("/home/kbj/Downloads/station_with_dong_only.csv")

df["행정동_only"] = df["행정동_only"].str.replace("·", ".", regex=False)

df.to_csv("station_with_dong_only_fixed.csv", index=False)

print(df[["행정동_only"]].head())

  행정동_only
0      서교동
1     자양3동
2     신정7동
3     신월3동
4      목1동


In [ ]:
import pandas as pd
import mysql.connector

# CSV 불러오기
df = pd.read_csv("/home/kbj/Downloads/station_with_dong_only_fixed.csv")

# MySQL 연결
conn = mysql.connector.connect(
    host="host",
    port=0000,
    user="user",
    password="password",
    database="project1"
)
cursor = conn.cursor()

# 테이블 생성
create_table_sql = """
CREATE TABLE IF NOT EXISTS station_b (
    station_id VARCHAR(20) PRIMARY KEY,
    station_gu VARCHAR(20),
    station_latitude DOUBLE,
    station_longitude DOUBLE,
    station_elevation INT,
    station_dong VARCHAR(50)
);
"""
cursor.execute(create_table_sql)

# 데이터 삽입
for _, row in df.iterrows():
    cursor.execute("""
        INSERT INTO station_b (station_id, station_gu, station_latitude, station_longitude, station_elevation, station_dong)
        VALUES (%s, %s, %s, %s, %s, %s)
    """, (
        row['station_id'],
        row['station_gu'],
        row['station_latitude'],
        row['station_longitude'],
        int(row['station_elevation']),
        row['행정동_only']  # 여기 수정됨!
    ))

conn.commit()
cursor.close()
conn.close()
